# TOX

Tox es una herramienta de manejo de entornos virtuales que nos va a permitir integrar todas las cosas que hemos visto hasta ahora:

- Conseguir estandar de pep8: flake8 + black
- Testear codigo desarrollado: pytest
- Medir la cobertura de los test: coverage.py + pytest-cov
- Crear paquetes de python: pyproject.toml
- Declarar archivos fuente del proyecto: MANIFEST.in
- Crear entornos virtuales : virtualenv

## Instalacion
- pip install tox

Para los test por ejemplo cada vez que vamos a correr distintas configuraciones de nuestro test, crea un entorno virtual nuevo, instala las dependencias de nuestro proyecto en ese entorno virtual y ejectua los test en ese entorno. Es decir, crea un entorno vritual para probar nuestro proyecto.

## Resumen: Como se trabaja con tox

- Creamos un archivo tox.ini
- El archivo consta de dos secciones importantes:
    - [tox]
    - [testenv:entorno1]
    - [testenv:entorno2]
    - [testenv:entorno3]
    - etc
- En [tox] agregamos los entornos creados
    - envlist = 
        - entorno1,
        - entorno2,
        - entorno3,
- En lo mas basico cada [testenv:entorno] consta de:
    - description : Una descripcion que describe lo que hace el entorno
    - skip_install : default=false *Skip installation of the package. This can be used when you need the virtualenv management but do not want to install the current package into that environment*
    - deps: Las dependencias que se requieren instalar en el entorno
    - commands: Los comandos que debe ejecutar por defecto cada entorno, al final de estos agregamos siempre {posargs} para que el usuario pueda agregar mas comandos si quiere al utilizar el entorno
    - usedevelop: default=false permite el modo editable -e al instalar un paquete, esto significaba que si vos hacias un cambio en la sourcefiles de un paquete este cambio es dinamico y se refleja en la instalacion. En el siguiente link de stack overflow se discute eso y sirve para mas info <a href='https://stackoverflow.com/questions/74789171/what-is-use-develop-in-tox-and-development-mode'>LINK</a>

- Como nota al final, si vos estas corriendo un solo entorno: **tox -e entorno** podes agregar comando simplemente agregando mas flags, siempre y cuando este entorno tenga especificado {posargs} en la seccion de commands, ejemplo: **tox -e entorno --flags**


## Archivo de configuracion de tox

*Pedimos que se pruebe nuestro paquete en entornos con python 3.9 y 3.7, tambien pide crear los entornos de style y coverage y otro para testear el check-manifest*<br>
*Pero concretamente que es esto?, abajo, cuando declaremos envlist, tenes que poner toda la lista de entornos que creaste. Para crear un entorno tenes que usar la sintaxis [testenv:nombre] , le agregas sus dependencias y comandos y luego tenes que agregar **nombre** a la lista de envlist*


<div style='color:green'>[tox]</div>
<div><p style='color:white'>envlist = py39, py37, style, coverage, check-manifest</p></div>

*Se declara el entorno global, no tiene nombre, todo aquello que no se ejecute en los entornos de arriba se ejecuta en este entorno*
Aqui {posargs} es para enviar comandos al entorno subyacente, por ejemplo indicar si queremos test mas verbosos menos verbosos, etc.
<div style='color:green'>[testenv]</div>
<div><p style='color:white'>deps = pytest</p></div>
<div><p style='color:white'>commands = pytest tests/ {posargs}</p></div>

*Aqui estamos haciendo los settings de los entornos virtuales que ya habiamos programado*
<div style='color:white'><div style='color:green'>[testenv:style]</div><br>
deps =<br>
flake8<br>
flake8-black<br>
flake8-isort<br>
commands = flake8 proyectox/ tests/ {posargs}</div>

*Lo mismo sucede con los settings de coverage y manifest*
<div style='color:white'><div style='color:green'>[testenv:coverage]</div><br>
deps =<br>
    coverage<br>
    pytest-cov<br>
commands =<br>
    pytest tests/ --cov=proyectox/ --cov-append --cov-report=term-missing<br>
    --fail-under=80</div>
<br>
<div style='color:white'><div style='color:green'>[testenv:check-manifest]</div><br>
deps = check-manifest<br>
commands = check-manifest</div>

### Ejecucion

Para ejecutar tox simplente escribimos:

- tox : ejecuta todo
- tox -e py39 : ejecuta solo el entorno de python 3.9 , -e hace referencia al enviroment que quiero ejecutar. Por otro lado si lo que queremos hacer es pasar los {posargs} tenemos que pasarlos como flags:<br>
    tox -e py39 -- **Comando**<br>
    tox -e py38 -- -m 'not slow'


## Consideraciones adicionales

Link al ejemplo del libro: <a href='https://github.com/daneah/publishing-python-packages/blob/main/ch05/first-python-package/setup.cfg'>Link</a>

Si yo corro el comando tox, la aplicacion escaneara por un archivo con la configuracion, de no encontrarlo nos tirara un error:

- ERROR: tox config file (either pyproject.toml, tox.ini, setup.cfg) not found

Tal y como se puede ver, es posible incluir la configuracion de tox en setup.cfg y pyproject.toml ademas del archivo tox.ini. Veremos como setear la configuracion en setup.cfg

Lo primero es agregar

<div style='color:red'>[tox:tox]</div>
<div style='color:white'>isolated_build = True</div>

*La linea anterior le idica a tox que construya nuestro paquete con los estandares de 517 y 518*

<div style='color:red'>[tox:tox]</div>
...
<div style='color:white'>envlist = py310</div>

*En esta seccion vamos a incluir todos los entornos que queremos que se creen. Recordar que podemos correr cada entorno de forma individual utilizando el comando:*

tox -e **Nombre del entorno**

*Por otro lado, tener en cuenta que en cada entorno, tox instala las independencias correspondientes cuando dicho entorno no las tenga ya instaladas.*

*Para nuestros tests tambien vamos a necesitar un entorno y particularmente le vamos a tener que indicar a tox que instale ciertas dependencias como pytest-cov*

<div style='color:red'>[testenv]</div>
...
<div style='color:white'>deps =</div>
<div style='color:white'>pytest</div>
<div style='color:white'>pytest-cov</div>
<div style='color:red'>commands =</div>
<div style='color:white'>    pytest {posargs}</div>

## Entornos a generar en tox

Aqui voy a poner una lista de entornos a generar en tox

[testenv:coverage] : Para el coverage de los test<br>
[testenv:style] : Para el estilo de los archivos<br>
[testenv:docstyle] : Para la documentacion<br>
[testenv:check-manifest] : Para la verificacion de Manifest.in<br>


Y ahora voy a pasar a hablar mas especificamente de estos dos, que no son tan obligatorios pero igual sirven:<br>

# check-headers

<div style='color:red'>[testenv:check-headers]</div><br>
skip_install = True<br>
deps =<br>
    https://github.com/quatrope/qafan/archive/refs/heads/master.zip<br>
commands =<br>
    check-headers check skcriteria/  tests/  --header-template .header-template {posargs}<br>

Este basicamente nos sirve para verificar un header que estemos utilizando en cada uno de nuestros archivos de python del proyecto.

Para utilizarlo necesitamos: 

- Crear el archivo .header-template
- Agregar un header como el siguiente:
    -<div style='color:white'> #!/usr/bin/env python <br>
        # -*- coding: utf-8 -*-<br>
        # License: BSD-3 (https://tldrlegal.com/license/bsd-3-clause-license-(revised))<br>
        # Copyright (c) 2016-2021, Cabral, Juan; Luczywo, Nadia<br>
        # Copyright (c) 2022, 2023, QuatroPe<br>
        # All rights reserved. </div>
- Para verficiar que este header esta en todos nuestros archivos utilizamos el comando:
    - check-headers check --header-template .header-template carpetadeinteres/ carpetadeinteres2/ ... etc
    - Este comando debe estar en tox porsupuesto
- Para agregarlo a tox en su correspondiente entorno virtual tenemos que tener en cuenta que utiliza las siguientes dependencias:
    - https://github.com/quatrope/qafan/archive/refs/heads/master.zip
    - Ese link es de un proyecto de juan, podes descargar el mismo e instalarlo en tu entorno virtual

# check-testdir 
Este es un toque mas facil, no hace falta crear ninigun archivo de configuracion, escanea simplemente tu directorio de proyecto vs tu directorio de tests y verifica si para archivo de tu proyecto existe un test con el mismo nombre.

Por regla general si tenemos un archivo llamado archivo.py debemos tener un test de nombre test_archivo.py.

El comando a ejecutar es el siguiente:

- check-testdir check carpetatests/ --reference-dir carpetaproyecto/
- Este comando debe estar en tox porsupuesto